<a href="https://colab.research.google.com/github/robertosolano/robertosolano/blob/master/Roberto_Experian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install Pyforest libraries to easy things up...**

In [ ]:
pip install --upgrade pyforest

  Preparing metadata (setup.py) ... done
  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14605 sha256=269d49f60084c098304f69e3d35e49b2c594d7e3b67ed933f9c99f0f7d1c0527
  Stored in directory: /root/.cache/pip/wheels/9e/7d/2c/5d2f5e62de376c386fd3bf5a8e5bd119ace6a9f48f49df6017
Successfully built pyforest


In [ ]:
%%python -m pyforest install_extensions

Starting to install pyforest extensions for Jupyter Notebook and Jupyter Lab

Trying to install pyforest nbextension...

Finished installing the pyforest Jupyter Notebook nbextension
Please reload your Jupyter notebook browser window

Trying to install pyforest labextension...
Could not install pyforest Jupyter Lab extension because Jupyter Lab is not available

Finished installing the pyforest Jupyter extensions
Please reload your Jupyter notebook and/or Jupyter lab browser windows


In [ ]:
import pyforest

# **Starting the excel file data import for analysis**

In [9]:
pyforest.active_imports()

import plotly as py
import pydot
from openpyxl import load_workbook
import plotly.express as px
import pandas as pd
import cv2
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import lightgbm as lgb


['import plotly as py',
 'import pydot',
 'from openpyxl import load_workbook',
 'import plotly.express as px',
 'import pandas as pd',
 'import cv2',
 'import pickle',
 'import matplotlib.pyplot as plt',
 'from sklearn.model_selection import cross_val_score',
 'import lightgbm as lgb']

In [14]:
# Employees: One record per employee. File Includes each employees User ID and  Office Location.
exp_employees_data = pd.read_excel('/content/Data Analyst Exercise Data (1).xlsx', sheet_name='Employees')
# User Licenses: One record per employee, per software licence. Includes User ID, Software Name, licence start and end date.
exp_user_licenses_data = pd.read_excel('/content/Data Analyst Exercise Data (1).xlsx', sheet_name='User Licences')
# User Access Log: One record per user log in. Includes User ID, Access Data, and Software Name
exp_user_access_data = pd.read_excel('/content/Data Analyst Exercise Data (1).xlsx', sheet_name='User Access Log')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
#Inspect Employees Data
exp_employees_data

,User ID,Office Location
0,2,UK
1,3,UK
2,4,UK
3,5,Brazil
4,6,US
...,...,...
66,94,UK
67,95,US
68,96,Brazil
69,98,UK


In [15]:
#Inspect User Access Log Data
exp_user_access_data

,User ID,Access Date,Software
0,54,2018-01-26,Tableau
1,76,2018-02-09,Alteryx
2,48,2018-02-21,Alteryx
3,20,2018-04-14,Tableau
4,52,2018-05-05,Tableau
...,...,...,...
89,71,2019-09-30,Tableau
90,92,2019-10-12,Tableau
91,2,2019-10-15,Tableau
92,16,2019-10-21,Tableau


## 1.	How many Employees are based at each Office Location?

In [18]:
# View available columns in table
exp_employees_data.columns

Index(['User ID', 'Office Location'], dtype='object')

In [19]:
# Select unique values from the Office Location column
exp_employees_data['Office Location'].unique()

array(['UK', 'Brazil', 'US'], dtype=object)

In [22]:
# Count the number of unique values per Office Location
exp_employees_data.groupby("Office Location")["User ID"].count()

# SQL Version for the above Python code
# SELECT state, count(name)
# FROM df
# GROUP BY state
# ORDER BY state;

Office Location
Brazil    21
UK        24
US        26
Name: User ID, dtype: int64

##2.	How many Employees in Brazil have an Alteryx licence?

In [130]:
employees_plus_licenses = pd.merge(exp_employees_data,exp_user_licenses_data,on='User ID')
employees_plus_licenses

<IPython.core.display.Javascript object>

,User ID,Office Location,Software,Licence Start Date,Licence End Date
0,2,UK,Tableau,2018-09-20,2019-09-19
1,3,UK,Alteryx,2018-08-31,2019-08-30
2,4,UK,Tableau,2018-12-31,2019-12-30
3,5,Brazil,Tableau,2018-07-30,2019-07-29
4,6,US,Tableau,2018-10-28,2019-10-27
...,...,...,...,...,...
95,95,US,Tableau,2018-10-21,2019-10-20
96,96,Brazil,Alteryx,2018-07-01,2019-06-30
97,96,Brazil,Tableau,2018-06-03,2019-06-02
98,98,UK,Tableau,2018-11-16,2019-11-15


In [32]:
query = employees_plus_licenses.groupby("Software")["User ID"].count()
query

Software
Alteryx    41
Tableau    59
Name: User ID, dtype: int64

In [54]:
filter1 = employees_plus_licenses["Office Location"].isin(["Brazil"])
filter2 = employees_plus_licenses["Software"].isin(["Tableau"])

# displaying data with both filters applied and mandatory
answer2 = employees_plus_licenses[filter1 & filter2]
answer2

,User ID,Office Location,Software,Licence Start Date,Licence End Date
3,5,Brazil,Tableau,2018-07-30,2019-07-29
7,10,Brazil,Tableau,2018-01-26,2019-01-25
17,20,Brazil,Tableau,2018-04-04,2019-04-03
19,22,Brazil,Tableau,2018-07-30,2019-07-29
22,28,Brazil,Tableau,2018-09-11,2019-09-10
24,30,Brazil,Tableau,2018-09-04,2019-09-03
32,35,Brazil,Tableau,2018-12-09,2019-12-08
42,46,Brazil,Tableau,2018-05-22,2019-05-21
56,57,Brazil,Tableau,2018-03-26,2019-03-25
64,69,Brazil,Tableau,2018-06-15,2019-06-14


In [55]:
# Answer: total of Brazil employees with Tableau license
answer2[filter1 & filter2].groupby("Software").count()

<ipython-input-55-28c3dca75278>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  answer2[filter1 & filter2].groupby("Software").count()


,User ID,Office Location,Licence Start Date,Licence End Date
Software,,,,
Tableau,18,18,18,18


In [66]:
# ANSWER
len(answer2)

18

## 3.	How many Employees have both a Tableau licence and an Alteryx licence?

In [247]:
filter3 = employees_plus_licenses.Software == 'Alteryx'

filter4 = employees_plus_licenses.Software == 'Tableau'

# displaying data with both filters applied and mandatory
#employees_plus_licenses[filter3 & filter4]

employees_plus_licenses["Duplicated"] = employees_plus_licenses.duplicated(subset=['User ID'])
#By default, for each set of duplicated values, the first occurrence is set on False and all others on True.
employees_plus_licenses.groupby("Duplicated")["User ID"].count()

Duplicated
False    71
True     29
Name: User ID, dtype: int64

## 4.	What is the earliest and latest Access Date for each software application?

In [154]:
full_data = pd.merge(employees_plus_licenses,exp_user_access_data,on='User ID')
full_data = full_data.drop(columns=['Software_y'])
full_data = full_data.rename(columns={'Software_x': 'Software'})
full_data

<IPython.core.display.Javascript object>

,User ID,Office Location,Software,Licence Start Date,Licence End Date,Access Date
0,2,UK,Tableau,2018-09-20,2019-09-19,2019-06-09
1,2,UK,Tableau,2018-09-20,2019-09-19,2019-06-20
2,2,UK,Tableau,2018-09-20,2019-09-19,2019-10-15
3,3,UK,Alteryx,2018-08-31,2019-08-30,2019-07-01
4,5,Brazil,Tableau,2018-07-30,2019-07-29,2018-10-03
...,...,...,...,...,...,...
142,96,Brazil,Alteryx,2018-07-01,2019-06-30,2018-07-07
143,96,Brazil,Alteryx,2018-07-01,2019-06-30,2018-10-23
144,96,Brazil,Tableau,2018-06-03,2019-06-02,2018-07-07
145,96,Brazil,Tableau,2018-06-03,2019-06-02,2018-10-23


In [184]:
query1 = full_data[full_data.Software.isin(['Tableau'])]
print("Tableau Latest Access")
query1['Access Date'].max()

Tableau Latest Access


Timestamp('2019-12-14 00:00:00')

In [185]:
print("Tableau Earliest Access")
query1['Access Date'].min()

Tableau Earliest Access


Timestamp('2018-01-26 00:00:00')

### Revisar esto de abajo algo esta raro...

In [188]:
query2 = full_data[full_data.Software.isin(['Alteryx'])]
print("Alteryx Latest Access")
query2['Access Date'].max()
# Excel dice 2019-08-30

Alteryx Latest Access


Timestamp('2019-09-30 00:00:00')

In [189]:
print("Alteryx Earliest Access")
query2['Access Date'].min()
# Excel dice 2018-02-09

Alteryx Earliest Access


Timestamp('2018-01-26 00:00:00')

## 5.	Use data analysis to identify if any software has been used inappropriately?

In [215]:
condition1 = np.where(full_data["Access Date"] > full_data["Licence End Date"],"true","false")

full_data["Violation"] = condition1
#full_data

,User ID,Office Location,Software,Licence Start Date,Licence End Date,Access Date,Violation
0,2,UK,Tableau,2018-09-20,2019-09-19,2019-06-09,false
1,2,UK,Tableau,2018-09-20,2019-09-19,2019-06-20,false
2,2,UK,Tableau,2018-09-20,2019-09-19,2019-10-15,true
3,3,UK,Alteryx,2018-08-31,2019-08-30,2019-07-01,false
4,5,Brazil,Tableau,2018-07-30,2019-07-29,2018-10-03,false
...,...,...,...,...,...,...,...
142,96,Brazil,Alteryx,2018-07-01,2019-06-30,2018-07-07,false
143,96,Brazil,Alteryx,2018-07-01,2019-06-30,2018-10-23,false
144,96,Brazil,Tableau,2018-06-03,2019-06-02,2018-07-07,false
145,96,Brazil,Tableau,2018-06-03,2019-06-02,2018-10-23,false


In [221]:
answer5 = full_data[full_data.Violation.isin(['true'])]
answer5

,User ID,Office Location,Software,Licence Start Date,Licence End Date,Access Date,Violation
2,2,UK,Tableau,2018-09-20,2019-09-19,2019-10-15,true
12,16,UK,Tableau,2018-09-21,2019-09-20,2019-10-21,true
31,33,US,Alteryx,2018-02-20,2019-02-19,2019-05-31,true
46,38,US,Tableau,2018-06-15,2019-06-14,2019-08-03,true
61,47,US,Alteryx,2018-01-27,2019-01-26,2019-07-13,true
63,47,US,Tableau,2018-05-31,2019-05-30,2019-07-13,true
78,54,UK,Tableau,2018-01-26,2019-01-25,2019-03-17,true
79,54,UK,Tableau,2018-01-26,2019-01-25,2019-07-13,true
83,56,UK,Tableau,2018-08-10,2019-08-09,2019-09-30,true
88,63,UK,Tableau,2018-03-12,2019-03-11,2019-03-23,true


In [222]:
# Count the number of violation values per Office Location
answer5.groupby("Office Location")["User ID"].count()

Office Location
Brazil     1
UK        15
US         6
Name: User ID, dtype: int64

22 licence violations...

In [224]:
len(answer5)

22